In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
from torch.nn import init
from torchtext.legacy import data
from torchtext.vocab import Vectors
from torchtext.vocab import GloVe
import time
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2021-03-25 11:43:54--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2021-03-25 11:43:54--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2021-03-25 11:43:55--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [ ]:
!unzip glove*.zip

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [ ]:
import numpy as np

print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.42B.300d.txt', encoding='utf-8') #need to be done...
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 1917494 word vectors.


In [ ]:
import pandas as pd

train = pd.read_csv('./traindata.csv', sep='\t', header= None)
train = train.loc[:, [4,2,0]]
train = train.rename(index=str, columns={ 0:"polarity" ,2: 'aspect',4: "text"})
stn = {'positive': 1,'neutral':0,'negative': -1}
train['polarity'] = train['polarity'].map(stn)
#train['review'] = train['review'].apply(lambda x: nltk.word_tokenize(x))
#dataset = dataset.rename(index=str, columns={ 0: "sentiment", 1: "aspect_category", 2: "review"})
train.head(5)

,text,aspect,polarity
0,short and sweet – seating is great:it's romant...,seating,1
1,This quaint and romantic trattoria is at the t...,trattoria,1
2,The have over 100 different beers to offer thi...,food,1
3,THIS STAFF SHOULD BE FIRED.,STAFF,-1
4,"The menu looked great, and the waiter was very...",menu,1


In [ ]:
val = pd.read_csv('./devdata.csv', sep='\t', header= None)
val = val.loc[:, [4,2,0]]
val = val.rename(index=str, columns={0:"polarity" ,2: 'aspect',4: "text"})
val['polarity'] = val['polarity'].map(stn)
#val['review']=val['review'].apply(lambda x: nltk.word_tokenize(x))
#dataset = dataset.rename(index=str, columns={ 0: "sentiment", 1: "aspect_category", 2: "review"})
val.head(5)

,text,aspect,polarity
0,"great food, great wine list, great service in ...",neighborhood,1
1,I thought this place was totally overrated.,place,-1
2,Fish is so very fresh.,Fish,1
3,"I showed it to the manager, and he smilingly a...",manager,-1
4,"The food we ordered was excellent, although I ...",margaritas,0


In [ ]:
train.to_csv('train.csv')
val.to_csv('val.csv')

In [ ]:
from nltk.tokenize import word_tokenize

label = data.Field(sequential=False)
aspect = data.Field(sequential=False,lower=True)
text = data.Field(sequential=True,lower=True,tokenize=word_tokenize)

In [ ]:
train, val = data.TabularDataset.splits(path='./',
                                        skip_header=True,
                                        train='train.csv',
                                        validation='val.csv',
                                        format='csv',
                                        fields=[('',None),('text', text),('aspect', aspect),('polarity', label)
                                                
                                                ])

In [ ]:
vectors = Vectors(name='glove.42B.300d.txt') #need to be done...

label.build_vocab(train, val)
aspect.build_vocab(train, val, vectors=vectors)
text.build_vocab(train, val, vectors=vectors)


text_vocab_size = len(text.vocab)
aspect_vocab_size = len(aspect.vocab)
text_vector=text.vocab.vectors
aspect_vector=aspect.vocab.vectors

100%|█████████▉| 1916818/1917494 [04:32<00:00, 7285.03it/s]

In [ ]:
batch_size=128
train_iter, val_iter = data.Iterator.splits(
            (train, val),
            sort_key=lambda x: len(x.text),
            batch_sizes=(batch_size, len(val)) # 训练集设置batch_size,验证集整个集合用于测试
    )                            

In [ ]:
class ATAE_LSTM(nn.Module):
    def __init__(self, embedding_dim, num_hiddens, num_layers):
        super(ATAE_LSTM, self).__init__()
        self.text_embeddings = nn.Embedding(text_vocab_size, embedding_dim)
        self.aspect_embeddings = nn.Embedding(aspect_vocab_size, embedding_dim)
        self.text_embeddings = nn.Embedding.from_pretrained(text_vector,
                                                            freeze=False)
        self.aspect_embeddings = nn.Embedding.from_pretrained(aspect_vector,
                                                              freeze=False)
        self.lstm = nn.LSTM(input_size=2 * embedding_dim,
                            hidden_size=num_hiddens,
                            num_layers=num_layers,
                            batch_first=True,
                            bidirectional=True)
        self.wh = nn.Parameter(torch.Tensor(num_hiddens * 2, num_hiddens * 2))
        self.wv = nn.Parameter(torch.Tensor(embedding_dim, embedding_dim))
        self.omega = nn.Parameter(
            torch.Tensor(1, embedding_dim*2))
        self.wp = nn.Parameter(torch.Tensor(num_hiddens * 2, num_hiddens * 2))
        self.wx = nn.Parameter(torch.Tensor(num_hiddens * 2, num_hiddens * 2))
        self.ws = nn.Parameter(torch.Tensor(4, num_hiddens * 2))
        nn.init.uniform_(self.wh, -0.1, 0.1)
        nn.init.uniform_(self.wv, -0.1, 0.1)
        nn.init.uniform_(self.omega, -0.1, 0.1)
        nn.init.uniform_(self.wp, -0.1, 0.1)
        nn.init.uniform_(self.wx, -0.1, 0.1)
        nn.init.uniform_(self.ws, -0.1, 0.1)
        self.bs = nn.Parameter(torch.zeros((4, 1)))

    def forward(self, text, aspect):
        seq_len = len(text.t())
        e1 = self.text_embeddings(text)
        # e1 形状是(batch_size,seq_len, embedding_dim)
        e2 = self.aspect_embeddings(aspect).expand(e1.size())

        wv = torch.cat((e1, e2), dim=2)
        # e.g.
        # wv torch.Size([batch_size,seq_len,2*embedding_dim])

        out, (h, c) = self.lstm(wv)  # output, (h, c)
        # out形状是(batch_size,seq_len, 2 * num_hiddens)
        # h形状是(num_layers * num_directions, batch_size, 2*num_hiddens)

        H = out.permute(0, 2, 1)
        # H形状是(batch_size,2 * num_hiddens,seq_len)
        #print(H.shape)
        #print(self.wh.shape)

        Wh_H = torch.matmul(self.wh, H)
        # wh 形状是(2*num_hiddens, 2*num_hiddens)
        # wh_H 形状是(batch_size, 2*num_hiddens, seq_len)
        #print('Wh_H: ', Wh_H.shape)
        Wv_Va_eN = torch.matmul(
            self.wv,
            self.aspect_embeddings(aspect).permute(0, 2, 1).expand(
                -1, embedding_dim, seq_len))
        # Wv 形状是(seq_len, seq_len) embedding_dim=2*num_hiddens
        # Wv_Va_eN 形状是(batch_size, embedding_dim, seq_len)
        #print('Wv_Va_eN: ', Wv_Va_eN.shape)

        vh = torch.cat((Wh_H, Wv_Va_eN), dim=1)
        # vh 形状是(batch_size, 2*embedding_dim, seq_len)
        #print('vh: ', vh.shape)

        M = torch.tanh(vh)
        # M 形状是(batch_size, 2*embedding_dim, seq_len)
        #print('M: ', M.shape)

        alpha = F.softmax(torch.matmul(self.omega, M),dim=2)
        # omega 形状为(1, 2*embedding_dim))
        # alpha 形状为(batch_size, 1, seq_len)
        #print('alpha: ', alpha.shape)

        r = torch.matmul(H, alpha.permute(0, 2, 1))
        # H形状是(batch_size,2 * num_hiddens,seq_len)
        # r 形状为(batch_size,2*num_hiddens,1)
        #print('r: ', r.shape)

        h_star = torch.tanh(
            torch.matmul(self.wp, r) +
            torch.matmul(self.wx, torch.unsqueeze(H[:, :, -1], 2)))
        # h_star形状是(batch_size,2 * num_hiddens,1)
        #print('h_star: ', h_star.shape)

        y = torch.matmul(self.ws, h_star) + self.bs  #不需要手动求softmax
        # y 形状(batch_size,4,1)
        
        y = y.reshape([-1, 4])
        # y 形状(batch_size,4)
        # ws 形状(4, 2*num_hiddens)
        #print('y: ', y.shape)

        return y

In [ ]:
embedding_dim, num_hiddens, num_layers = 300, 150, 3
net = ATAE_LSTM(embedding_dim, num_hiddens, num_layers)
print(net)

ATAE_LSTM(
  (text_embeddings): Embedding(2760, 300)
  (aspect_embeddings): Embedding(676, 300)
  (lstm): LSTM(600, 150, num_layers=3, batch_first=True, bidirectional=True)
)


In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_iter):
            X1, X2, y = batch.text, batch.aspect, batch.polarity
            X1 = X1.permute(1, 0)
            X2 = X2.unsqueeze(1)
            if isinstance(net, torch.nn.Module):
                net.eval()  # 评估模式, 这会关闭dropout
                acc_sum += (net(X1,
                                X2).argmax(dim=1) == y).float().sum().item()
                net.train()  # 改回训练模式
            else:
                if ('is_training'
                        in net.__code__.co_varnames):  # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X1, X2, is_training=False).argmax(
                        dim=1) == y).float().sum().item()
                else:
                    acc_sum += (net(
                        X1, X2).argmax(dim=1) == y).float().sum().item()
            n += y.shape[0]
    return acc_sum / n

In [ ]:
def train(train_iter, test_iter, net, loss, optimizer, num_epochs):
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for batch_idx, batch in enumerate(train_iter):
            X1, X2, y = batch.text, batch.aspect, batch.polarity
            X1 = X1.permute(1, 0)
            X2 = X2.unsqueeze(1)
            y_hat = net(X1,X2)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print(
            'epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
            % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n,
               test_acc, time.time() - start))

In [ ]:
lr, num_epochs = 0.01, 20
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
train(train_iter, val_iter, net, loss, optimizer, num_epochs)

epoch 1, loss 1.3231, train acc 0.605, test acc 0.702, time 35.8 sec
epoch 2, loss 0.4068, train acc 0.653, test acc 0.702, time 36.4 sec
epoch 3, loss 0.2526, train acc 0.655, test acc 0.702, time 40.1 sec
epoch 4, loss 0.1763, train acc 0.687, test acc 0.702, time 36.4 sec
epoch 5, loss 0.1276, train acc 0.706, test acc 0.705, time 33.5 sec
epoch 6, loss 0.0975, train acc 0.777, test acc 0.713, time 33.0 sec
epoch 7, loss 0.0614, train acc 0.857, test acc 0.721, time 33.8 sec
epoch 8, loss 0.0430, train acc 0.894, test acc 0.750, time 34.6 sec
epoch 9, loss 0.0321, train acc 0.917, test acc 0.747, time 32.7 sec
epoch 10, loss 0.0253, train acc 0.925, test acc 0.766, time 31.8 sec
epoch 11, loss 0.0214, train acc 0.923, test acc 0.793, time 33.3 sec
epoch 12, loss 0.0172, train acc 0.931, test acc 0.723, time 33.8 sec
epoch 13, loss 0.0158, train acc 0.937, test acc 0.718, time 32.3 sec
epoch 14, loss 0.0124, train acc 0.942, test acc 0.715, time 34.9 sec
epoch 15, loss 0.0106, train 